In [3]:
import glob
from pathlib import Path

import geopandas as gpd
import ipywidgets as widgets

import matplotlib.pyplot as plt
import rasterio
from ipyleaflet import (
    FullScreenControl,
    GeoData,
    LayersControl,
    LegendControl,
    Map,
    Marker,
    ScaleControl,
    SplitMapControl,
    WidgetControl,
    basemaps,
    Popup,
)
from IPython.display import Markdown, display
from ipywidgets import Button, IntSlider, Layout, jslink, widgets, HTML
from localtileserver import TileClient, get_leaflet_tile_layer
from scipy.ndimage import gaussian_filter
import numpy as np
from osgeo import gdal
from pathlib import Path

In [ ]:
import rasterio
def get_coordinate_pixel(tiff_file, lat, lon):
    dataset = rasterio.open(tiff_file)
    py, px = dataset.index(lon, lat)
    # create 1x1px window of the pixel
    window = rasterio.windows.Window(px - 1 // 2, py - 1 // 2, 1, 1)
    # read rgb values of the window
    clip = dataset.read(window=window)
    return clip[0][0][0]

In [6]:
def getCoordinatePixel(maps, lon, lat):
    # open map
    dataset = rasterio.open(maps)
    # get pixel x+y of the coordinate
    py, px = dataset.index(lon, lat)
    # print(py, px)
    # create 1x1px window of the pixel
    window = rasterio.windows.Window(px - 1 // 2, py - 1 // 2, 1, 1)
    # read rgb values of the window
    clip = dataset.read(window=window)
    return clip[0][0][0]


lat = -3.7957
long = -80.1151
max_zoom = 30


def addElOro(map_obj, selShp):
    selDf = gpd.read_file("Ecuador shapefiles/" + selShp + ".shp")
    selDf = selDf[selDf["ADM1_ES"] == "El Oro"]

    geoDf = selDf.to_crs(4326)
    lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean() / 2
    latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean() / 2

    map_obj.center = (latCent, lonCent)
    geoData = GeoData(
        geo_dataframe=geoDf,
        name=selShp,
        style={"color": "magenta", "fillColor": "magenta", "fillOpacity": 0.05},
    )  # style={'color': 'black', 'fillColor': 'green', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    # hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
    map_obj.add_layer(geoData)


def addEcuador(map_obj):
    selDf = gpd.read_file("Ecuador shapefiles/ecu_admbnda_adm0_inec_20190724.shp")
    selDf = selDf[selDf["ADM0_EN"] == "Ecuador"]

    geoDf = selDf.to_crs(4326)
#     lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean() / 2
#     latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean() / 2

#     map_obj.center = (latCent, lonCent)
    geoData = GeoData(
        geo_dataframe=geoDf,
        name="ecu_admbnda_adm0_inec_20190724",
        style={"color": "pink", "fillColor": "magenta", "fillOpacity": 0},
    )  # style={'color': 'black', 'fillColor': 'green', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    # hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
    map_obj.add_layer(geoData)


def get_shapefile_bounds(file_path, column="ADM0_EN", value="Ecuador"):
    # Load the shapefile
    gdf = gpd.read_file(file_path)

    # Filter the DataFrame for the specified region
    gdf = gdf[gdf[column] == value]

    # Calculate the bounds for the region
    # bounds = gdf.bounds

    # If you want to transform the bounds to WGS84 (EPSG:4326), you can do:
    gdf = gdf.to_crs("EPSG:4326")
    bounds = gdf.total_bounds
    minX, minY, maxX, maxY = bounds
    bounds_tuple = (minX, maxY, maxX, minY)
    # [[south, west], [north, east]]
    # Return the bounds
    return bounds_tuple


def smooth_dataset(file_path, sigma):
    path = Path(file_path)
    smoothed_file = path.parent / f"{path.stem}_smoothed_{sigma}_{path.suffix}"
    if smoothed_file.exists():
        return rasterio.open(smoothed_file)
    # Open the file
    with rasterio.open(file_path) as src:
        # Create an empty list to store the smoothed bands
        smoothed_bands = []

        # Apply the Gaussian filter to each band
        for band in src.read():
            smoothed_bands.append(gaussian_filter(band, sigma=sigma))

        # Create a new GeoTiff file

        with rasterio.open(
            smoothed_file,
            "w",
            driver="GTiff",
            height=src.height,
            width=src.width,
            count=src.count,
            dtype=src.dtypes[0],
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            # Write the smoothed bands to the new file
            for i, band in enumerate(smoothed_bands, start=1):
                dst.write(band, i)

    # Return a new DatasetReader for the new file
    return rasterio.open(smoothed_file)


# +



def crop_and_reproject_dataset(
    file_path, bbox, xRes=None, yRes=None, resample_alg=gdal.GRA_CubicSpline
):
    path = Path(file_path)
    cropped_file = str(path.parent / f"{path.stem}_cropped.tif")
    reprojected_file = str(path.parent / f"{path.stem}_interpolated.tif")
    if Path(reprojected_file).exists():
        return rasterio.open(reprojected_file)
    dst_crs = "EPSG:4326"
    # Open the source file
    src_ds = gdal.Open(str(file_path))
    if src_ds is None:
        print(f"Could not open {file_path}")
        return None

    # Crop the source image
    translate_options = gdal.TranslateOptions(projWin=bbox)
    cropped_ds = gdal.Translate(cropped_file, src_ds, options=translate_options)

    # Clean up
    src_ds = None

    # Create Warp options
    # srcNodata=0
    warp_options = gdal.WarpOptions(
        dstSRS=dst_crs, xRes=xRes, yRes=yRes, resampleAlg=resample_alg, srcNodata=0, dstNodata=0
    )

    # Reproject cropped image
    reprojected_ds = gdal.Warp(reprojected_file, cropped_ds, options=warp_options)

    # Clean up
    cropped_ds = None
    reprojected_ds = None  # Close the dataset to write to disk
    Path(cropped_file).unlink()
    return rasterio.open(reprojected_file)


def smooth(file_path):
    minx = -82.0
    maxy = -6.00
    maxx = -77.0
    miny = 1.2  # -1.00
    bbox = (minx, miny, maxx, maxy)
    # bbox = get_shapefile_bounds("Ecuador shapefiles/ecu_admbnda_adm0_inec_20190724.shp")
    # print(bbox)
    bbox = (-82.00896662899999, 1.681834608000031, -75.18714655299993, -5.2)
    return crop_and_reproject_dataset(file_path, bbox, xRes=0.01, yRes=0.01)

In [7]:
# +
# production in 1000T
client = TileClient(smooth("Actual Yields and Production/T/2010/ban_2010_prd.tif"))
yield_display = dict(
    vmin=2,
    vmax=21,
    cmap="coolwarm",
    opacity=0.75,
    max_zoom=max_zoom,
    max_native_zoom=max_zoom,
)
# Create ipyleaflet tile layer from that server
t = get_leaflet_tile_layer(client, **yield_display)

actual_yield_map = Map(
    center=(lat, long),
    zoom=9,
    layout=Layout(height="800px"),
    basemap=basemaps.Esri.WorldImagery,
)

addEcuador(actual_yield_map)
addElOro(actual_yield_map, "ecu_admbnda_adm2_inec_20190724")
yieldd = getCoordinatePixel(
    "Actual Yields and Production/T/2010/ban_2010_prd.tif", long, lat
)
# Create a marker at the given latitude and longitude
marker = Marker(location=(lat, long))

def marker_move(event, location):
    message1 = HTML()
    pixel_value = getCoordinatePixel(
        "Actual Yields and Production/T/2010/ban_2010_prd.tif", location[1], location[0]
    )
    message1.value = (
        f"Yield: {pixel_value*1000:,.0f} tons at {location[0]:.4f}, {location[1]:.4f}"
    )

    # Popup with a given location on the map:
    popup = Popup(
        location=location,
        child=message1,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False,
    )
    actual_yield_map.add_layer(popup)
    marker.popup = message1


marker.on_move(marker_move)
# Display the widget

# Add the marker to the map
actual_yield_map.add_layer(marker)
actual_yield_map.add_control(ScaleControl(position="bottomleft"))
actual_yield_map.add_layer(t)
display(Markdown("## Actual Yield 2010"))
display(actual_yield_map)

## Actual Yield 2010

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [8]:
# +
file_name_mapping = {
    "mean_organic_carbon_density_0_5": "Mean Organic Carbon Density (0-5cm, g/dm³)",
    "mean_soil_organic_carbon_0_5": "Mean Soil Organic Carbon (0-5cm, dg/kg)",
    "mean_silt_0_5": "Mean Silt (0-5cm, g/kg)",
    "mean_clay_content_0_5": "Mean Clay Content (0-5cm, g/kg)",
    "mean_nitrogen_0_5": "Mean Nitrogen (0-5cm, cg/kg)",
    "mean_course_fragments_0_5": "Mean Course Fragments (0-5cm, cm³/dm³)",
    "mean_cation_exchange_capacity_0_5": "Mean Cation Exchange Capacity (0-5cm, mmol(c)/kg)",
    "mean_sand_0_5": "Mean Sand (0-5cm, g/kg)",
    "mean_bulk_density_0_5": "Mean Bulk Density (0-5cm, cg/cm³)",
    "mean_organic_carbon_stock_0_30": "Mean Organic Carbon Stock (0-30cm, tonnes/ha)",
    "world_reference_soil_groups": "World Reference Soil Groups",
    "mean_ph_water_0_5": "Mean pH Water (0-5cm, pH * 10)",
}

for file in glob.glob("Soil Grids/*.tif"):
    file_p = Path(file)
    # production in 1000T
    client = TileClient(file)
    # Create ipyleaflet tile layer from that server
    cmap = "coolwarm"
    if file_p.stem == "world_reference_soil_groups":
        cmap = "tab20"
        legend = LegendControl(
            {
                "Cambisols": "Green",
                "Ferralsols": "#d62728",
                "Andosols": "Orange",
                "Fluvisols": "rgb(231,150,142)",
                "Gleysols": "rgb(118, 96, 146)",
            },
            name="Legend",
            position="topright",
        )

        # Add the legend to the map

    t = get_leaflet_tile_layer(
        client,  # vmin=0, vmax=50,
        cmap=cmap,
        opacity=0.75,
        max_zoom=max_zoom,
        max_native_zoom=max_zoom,
    )

    m = Map(
        center=(lat, long),
        zoom=10,
        layout=Layout(height="800px"),
        basemap=basemaps.Esri.WorldImagery,
    )
    addElOro(m, "ecu_admbnda_adm2_inec_20190724")

    # Create a marker at the given latitude and longitude
    pix_value = getCoordinatePixel(
        file,
        long,
        lat,
    )
    marker = Marker(location=(lat, long), title=f"Value: {pix_value}")
    if file_p.stem == "world_reference_soil_groups":
        m.add_control(legend)
    # Add the marker to the map
    m.add_layer(marker)
    m.add_layer(t)

    display(Markdown(f"## {file_name_mapping[file_p.stem]}"))
    display(m)

## Mean Soil Organic Carbon (0-5cm, dg/kg)

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Mean Silt (0-5cm, g/kg)

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Mean Clay Content (0-5cm, g/kg)

...

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## World Reference Soil Groups

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Mean pH Water (0-5cm, pH * 10)

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [11]:
# -

# # Mean temperature animation

# +
display_dict = dict(
    vmin=6,
    vmax=30,
    cmap="coolwarm",
    opacity=0.75,
    max_zoom=max_zoom,
    max_native_zoom=max_zoom,
)
year_map = Map(
    center=(lat, long), zoom=9, basemap=basemaps.OpenStreetMap.Mapnik, dragging=False, layout=Layout(height="800px")
)

addElOro(year_map, "ecu_admbnda_adm2_inec_20190724")
def get_split_control(year):
    l_name = f"Agroclimatic/Thermal regime/Mean annual temp/ts_tmp_GFDL-ESM2M_rcp2p6_{year}.tif"
    r_name = f"Agroclimatic/Thermal regime/Mean annual temp/ts_tmp_GFDL-ESM2M_rcp8p5_{year}.tif"
    l_client = TileClient(l_name)
    r_client = TileClient(r_name)
    # Create new layers
    left_layer_new = get_leaflet_tile_layer(l_client, **display_dict)
    right_layer_new = get_leaflet_tile_layer(r_client, **display_dict)

    # Add a new SplitMapControl with the new layers
    control = SplitMapControl(left_layer=left_layer_new, right_layer=right_layer_new)
    return control


# Create a dictionary of GeoJSON layers, one for each unique timestamp
layers = {year: get_split_control(year) for year in range(2011, 2100)}

# Create a time slider
time_slider = widgets.SelectionSlider(
    options=list(layers.keys()), description="Year", continuous_update=False
)

# Create a Play widget
play = widgets.Play(
    interval=500,  # ms
    value=0,
    min=0,
    max=len(time_slider.options) - 1,
    step=1,
    description="Press play",
    disabled=False,
)

# Create an IntSlider that will be linked to the Play widget and the SelectionSlider
int_slider = widgets.IntSlider(min=0, max=len(time_slider.options) - 1)


def update_temp_label(event, location):
    year = time_slider.value
    m_lat, m_long = location
    l_name = f"Agroclimatic/Thermal regime/Mean annual temp/ts_tmp_GFDL-ESM2M_rcp2p6_{year}.tif"
    r_name = f"Agroclimatic/Thermal regime/Mean annual temp/ts_tmp_GFDL-ESM2M_rcp8p5_{year}.tif"
    l_value = getCoordinatePixel(l_name, m_long, m_lat)
    r_value = getCoordinatePixel(r_name, m_long, m_lat)
    rcps_value.value = f"Mean annual temperature RCP2.6 vs RCP8.5 at marker: {l_value:.0f} vs {r_value:.0f} °C"


# Function to update the map based on the time slider
def update_temp_map(change):
    # Remove all current layers
    for control in year_map.controls:
        if isinstance(control, SplitMapControl):
            year_map.remove_control(control)
        if isinstance(control, Popup):
            control.close_popup()
            year_map.remove_control(control)

    # Add the layer for the selected time
    year = time_slider.options[change["new"]]
    year_map.add_control(layers[year])
    update_temp_label(change, marker.location)


# Call update_map when the int_slider changes
int_slider.observe(update_temp_map, names="value")
temp_marker = Marker(location=(lat, long))
temp_marker.on_move(update_temp_label)

# Add the marker to the map
year_map.add_layer(temp_marker)


# Link the Play widget and the SelectionSlider to the IntSlider
widgets.jslink((play, "value"), (int_slider, "value"))
widgets.jslink((int_slider, "value"), (time_slider, "index"))
# Define a layout for the buttons
button_layout = Layout(width="50px")  # Or use percentage: '10%'

# Create the decrement button with a 'minus' icon and custom layout
dec_button = Button(
    icon="minus",  # 'minus' is the Font Awesome name for the minus icon
    button_style="",  # '' means default
    layout=button_layout,  # Use the custom layout
)

# Create the increment button with a 'plus' icon and custom layout
inc_button = Button(
    icon="plus",  # 'plus' is the Font Awesome name for the plus icon
    button_style="",  # '' means default
    layout=button_layout,  # Use the custom layout
)


# Define function for decrement button
def dec_value(b):
    if int_slider.value > int_slider.min:
        int_slider.value -= 1


# Define function for increment button
def inc_value(b):
    if int_slider.value < int_slider.max:
        int_slider.value += 1


# Link button click events to their respective functions
dec_button.on_click(dec_value)
inc_button.on_click(inc_value)
rcps_value = widgets.Label()
value = 6
cmap = plt.get_cmap("coolwarm")
norm = plt.Normalize(
    vmin=6, vmax=30
)  # Adjust the vmin and vmax values based on your data range

def color(value):
    return "#%02x%02x%02x" % tuple(int(255 * c) for c in cmap(norm(value))[:3])

legend_map = {f"{i}°C": color(i) for i in range(6, 31, 3)}
legend = LegendControl(legend_map, name="Legend", position="topright")
year_map.add_control(legend)
# Display the map and the widgets
year_map.add_control(ScaleControl(position="bottomleft"))
display(Markdown("## Mean Annual Temperature RCP2.6 vs RCP8.5"))
display(widgets.HBox([play, dec_button, time_slider, inc_button, rcps_value]))
display(year_map)
update_temp_map({"new": 0})

Map(center=[-3.7957, -80.1151], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [9]:
# +
# Create a list of years
years = ["2011-2040 (2020)", "2041-2070 (2050)", "2071-2100 (2080)"]
year_mapping = {
    "2071-2100 (2080)": "2080",
    "2011-2040 (2020)": "2020",
    "2041-2070 (2050)": "2050",
}
# Create a list of crops
crop_map = {
    "Banana": "bana",
    "Rice": "ricw",
    "Wheat": "whea",
    "Soybean": "soyb",
    "Sugarcane": "sugc",
}
# Create the widgets
year_widget = widgets.Dropdown(options=years, description="Select Year:")
crop_widget = widgets.Dropdown(options=crop_map.keys(), description="Select Crop:")

potential_yield_checkbox = widgets.Checkbox(
    value=False,
    description="Smoothed",
    disabled=False,
)
production_value = widgets.Label()
# Display the widgets
potential_yield_hbox = widgets.HBox(
    [year_widget, crop_widget, potential_yield_checkbox, production_value]
)

potential_yield_map = Map(
    center=(lat, long),
    zoom=8,
    dragging=False,
    layout=Layout(height="800px"),
    basemap=basemaps.Esri.WorldImagery,
)
zoom_slider = IntSlider(description="Zoom level:", min=0, max=15, value=9)
jslink((zoom_slider, "value"), (potential_yield_map, "zoom"))
potential_yield_widgetControl = WidgetControl(widget=zoom_slider, position="topright")


# Shared display parameters
potential_yield_display_dict = dict(
    # vmin=0, vmax=9100,
    cmap="coolwarm",
    opacity=0.75,
    max_zoom=max_zoom,
    max_native_zoom=max_zoom,
)

# Create 2 tile layers from different raster
potential_yield_l_client = TileClient(
    "Potential Yield/potential_yield_gfdlesm2m_rcp2p6_2020_bana200a_yld.tif"
)
potential_yield_r_client = TileClient(
    "Potential Yield/potential_yield_gfdlesm2m_rcp8p5_2020_bana200a_yld.tif"
)
left_layer_new = get_leaflet_tile_layer(
    potential_yield_l_client, **potential_yield_display_dict
)
right_layer_new = get_leaflet_tile_layer(
    potential_yield_r_client, **potential_yield_display_dict
)
potential_yield_control = SplitMapControl(
    left_layer=left_layer_new, right_layer=right_layer_new
)

potential_yield_marker = Marker(location=(lat, long))
potential_yield_map.add_layer(potential_yield_marker)


# Function to update map
def potential_yield_value_change(change):
    potential_yield_update_map(
        left_layer_new, right_layer_new, potential_yield_control, change
    )


def potential_yield_update_map(left_layer_new, right_layer_new, control, change):
    year = year_mapping[year_widget.value]
    crop = crop_map[crop_widget.value]
    smoothed = potential_yield_checkbox.value
    for control in potential_yield_map.controls:
        if isinstance(control, SplitMapControl):
            potential_yield_map.remove_control(control)
    l_name = (
        f"Potential Yield/potential_yield_gfdlesm2m_rcp2p6_{year}_{crop}200a_yld.tif"
    )
    r_name = (
        f"Potential Yield/potential_yield_gfdlesm2m_rcp8p5_{year}_{crop}200a_yld.tif"
    )
    if smoothed:
        l_name = smooth(l_name)
        r_name = smooth(r_name)
    l_client = TileClient(l_name)
    r_client = TileClient(r_name)
    # Create new layers
    left_layer_new = get_leaflet_tile_layer(l_client, **potential_yield_display_dict)
    right_layer_new = get_leaflet_tile_layer(r_client, **potential_yield_display_dict)

    # Add a new SplitMapControl with the new layers
    control = SplitMapControl(left_layer=left_layer_new, right_layer=right_layer_new)
    # Add the marker to the map
    potential_yield_map.add_control(control)
    potential_yield_marker_move(change, potential_yield_marker.location)


def potential_yield_marker_move(event, location):
    year = year_mapping[year_widget.value]
    crop = crop_map[crop_widget.value]
    new_lat, new_long = location
    l_pix = getCoordinatePixel(
        f"Potential Yield/potential_yield_gfdlesm2m_rcp2p6_{year}_{crop}200a_yld.tif",
        new_long,
        new_lat,
    )
    r_pix = getCoordinatePixel(
        f"Potential Yield/potential_yield_gfdlesm2m_rcp8p5_{year}_{crop}200a_yld.tif",
        new_long,
        new_lat,
    )
    production_value.value = f"Yield: {l_pix*1000:,.0f} vs {r_pix*1000:,.0f} tons/ha at {location[0]:.2f}, {location[1]:.2f}"


potential_yield_marker.on_move(potential_yield_marker_move)
# Set the function to be called on value change
year_widget.observe(potential_yield_value_change, names="value")
crop_widget.observe(potential_yield_value_change, names="value")
potential_yield_checkbox.observe(potential_yield_value_change, names="value")

addElOro(potential_yield_map, "ecu_admbnda_adm2_inec_20190724")
potential_yield_map.add_control(potential_yield_widgetControl)


# Create a marker at the given latitude and longitude

potential_yield_map.add_control(ScaleControl(position="bottomleft"))
potential_yield_map.add_control(LayersControl())
potential_yield_map.add_control(FullScreenControl())

# m.add_control(control)
display(Markdown("## Potential Crop Yield RCP2.6 vs RCP8.5"))
display(potential_yield_hbox)
display(potential_yield_map)
potential_yield_value_change({"new": 0})

## Potential Crop Yield RCP2.6 vs RCP8.5

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [10]:
# +
# Create a list of years
years = ["2011-2040 (2020)", "2041-2070 (2050)", "2071-2100 (2080)"]
year_mapping = {
    "2071-2100 (2080)": "2080",
    "2011-2040 (2020)": "2020",
    "2041-2070 (2050)": "2050",
}
year_vmap = {"2080": (2, 3500), "2050": (2, 400), "2020": (2, 70)}
# Create a list of crops

crop_map = {
    "Banana": "bana",
    "Rice": "ricw",
    "Wheat": "whea",
    "Soybean": "soyb",
    "Sugarcane": "sugc",
}
# Create the widgets
year_widget = widgets.Dropdown(options=years, description="Select Year:")
crop_widget = widgets.Dropdown(options=crop_map.keys(), description="Select Crop:")

yield_delta_checkbox = widgets.Checkbox(
    value=False,
    description="Smoothed",
    disabled=False,
)
yield_delta_value = widgets.Label()
# Display the widgets
yield_delta_hbox = widgets.HBox(
    [year_widget, crop_widget, yield_delta_checkbox, yield_delta_value]
)

yield_delta_map = Map(
    center=(lat, long),
    zoom=8,
    dragging=True,
    layout=Layout(height="800px"),
    basemap=basemaps.Esri.WorldImagery,
)
zoom_slider = IntSlider(description="Zoom level:", min=0, max=15, value=9)
jslink((zoom_slider, "value"), (yield_delta_map, "zoom"))
yield_delta_widgetControl = WidgetControl(widget=zoom_slider, position="topright")


# Shared display parameters
yield_delta_display_dict = dict(
    vmin=0,
    vmax=3000,
    cmap="coolwarm",
    opacity=0.75,
    max_zoom=max_zoom,
    max_native_zoom=max_zoom,
)

yield_delta_marker = Marker(location=(lat, long))
yield_delta_map.add_layer(yield_delta_marker)


# Function to update map
def yield_delta_value_change(change):
    yield_delta_update_map(change)


def yield_delta_update_map(change):
    year = year_mapping[year_widget.value]
    crop = crop_map[crop_widget.value]
    smoothed = yield_delta_checkbox.value
    yield_delta_display_dict["vmin"] = year_vmap[year][0]
    yield_delta_display_dict["vmax"] = year_vmap[year][1]
    for layer in yield_delta_map.layers:
        if "BoundTileLayer" in str(type(layer)):
            yield_delta_map.remove_control(layer)
    l_name = (
        f"Potential Yield/potential_yield_gfdlesm2m_rcp2p6_{year}_{crop}200a_yld.tif"
    )
    r_name = (
        f"Potential Yield/potential_yield_gfdlesm2m_rcp8p5_{year}_{crop}200a_yld.tif"
    )
    with rasterio.open(l_name) as src1:
        band1 = src1.read(1)  # , window=window)

    with rasterio.open(r_name) as src2:
        band2 = src2.read(1)  # , window=window)

    # Compute the delta
    delta = np.abs(band1 - band2)
    new_file = f"potential_yield_gfdlesm2m_DELTA_{year}_{crop}200a_yld.tif"
    with rasterio.open(
        new_file,
        "w",
        driver="GTiff",
        height=delta.shape[0],
        width=delta.shape[1],
        count=1,
        dtype=delta.dtype,
        crs=src1.crs,
        transform=src1.transform,
    ) as dst:
        dst.write(delta, 1)

    # Return a new DatasetReader for the new file
    if smoothed:
        delta_fs = smooth(new_file)
    else:
        delta_fs = rasterio.open(new_file)
    new_client = TileClient(delta_fs)
    # Create new layers
    layer_new = get_leaflet_tile_layer(new_client, **yield_delta_display_dict)

    # Add a new SplitMapControl with the new layers
    yield_delta_map.add_layer(layer_new)
    yield_delta_marker_move(change, marker.location)


def yield_delta_marker_move(event, location):
    year = year_mapping[year_widget.value]
    crop = crop_map[crop_widget.value]
    new_lat, new_long = location
    l_pix = getCoordinatePixel(
        f"Potential Yield/potential_yield_gfdlesm2m_rcp2p6_{year}_{crop}200a_yld.tif",
        new_long,
        new_lat,
    )
    r_pix = getCoordinatePixel(
        f"Potential Yield/potential_yield_gfdlesm2m_rcp8p5_{year}_{crop}200a_yld.tif",
        new_long,
        new_lat,
    )
    yield_delta_value.value = f"Yield: {l_pix*1000:,.0f} vs {r_pix*1000:,.0f} (delta {r_pix-l_pix}) tons/ha at {location[0]:.2f}, {location[1]:.2f}"


yield_delta_marker.on_move(yield_delta_marker_move)
# Set the function to be called on value change
year_widget.observe(yield_delta_value_change, names="value")
crop_widget.observe(yield_delta_value_change, names="value")
yield_delta_checkbox.observe(yield_delta_value_change, names="value")


yield_delta_map.add_control(yield_delta_widgetControl)


# Create a marker at the given latitude and longitude

yield_delta_map.add_control(ScaleControl(position="bottomleft"))
yield_delta_map.add_control(LayersControl())
yield_delta_map.add_control(FullScreenControl())
addElOro(yield_delta_map, "ecu_admbnda_adm2_inec_20190724")
display(Markdown("## Potential Crop Yield Delta RCP2.6 vs RCP8.5"))
display(yield_delta_hbox)
display(yield_delta_map)
yield_delta_value_change({"new": 0})

## Potential Crop Yield Delta RCP2.6 vs RCP8.5

Map(center=[-3.538806962642807, -79.84343566192852], controls=(ZoomControl(options=['position', 'zoom_in_text'…